# v1like demo

This notebook is to reproduce the results in the [v1like paper](http://dx.doi.org/10.1371/journal.pcbi.0040027) using the rewritten v1like module in this toolbox.

In [1]:
%matplotlib inline
from __future__ import print_function, division, absolute_import
import h5py
import numpy as np
import os
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from early_vision_toolbox.v1like import v1like
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from skimage.io import imread
from collections import Counter
from sklearn.multiclass import OneVsRestClassifier

use fftconvolve for legacy


In [2]:
# convert dataset to hdf5

img_path = '/home/leelab_share/datasets/Caltech_101/101_ObjectCategories/'
# code to get all filenames
tree = os.walk(img_path)


EXTENSIONS = ['.png', '.jpg']
filelist = []
label_list = []
categories = tree.next()[1]
for root, dirs, files in tree:
    assert not dirs, "must have no sub directory in each category!"
    filelist_this = [ root+'/'+f for f in files if os.path.splitext(f)[-1] in EXTENSIONS ]
    filelist += filelist_this
    label_list += [os.path.split(root)[1]]*len(filelist_this) 
num_class = len(categories)
assert num_class == 102

filelist = np.array(filelist)
label_list = np.array(label_list)
index_per_category = [np.flatnonzero(label_list==cat) for cat in categories]
label_counts = np.array(Counter(label_list).values())
print('mean count {}, std count {}, max count {}, min count {}'.format(label_counts.mean(), label_counts.std(),
                                                                      label_counts.max(), label_counts.min()))

# compute 10 folds to work on.
# here I simply used ShuffleSplit, instead of more correct StratifiedShuffleSplit, which has bug...
# < 
# sss = StratifiedShuffleSplit(label_list, 10, test_size=15*num_class, train_size=30*num_class, random_state=1)

# build a custom cv object that yields 30 test examples and 15 train examples for each class.

rng_state = np.random.RandomState(seed=0)
indices_per_category = []
for index_this in index_per_category:
    # use 15 test, since actually some classes don't have that many 30 examples...
    # in the original paper, they weighted each example acccording to number of exmaples per class.
    # but there's no such functionality in sklearn.
    ss_for_this_category = ShuffleSplit(len(index_this), n_iter=5, test_size=15, train_size=15, random_state=rng_state)
    indices_per_category.append(ss_for_this_category)

ss_all = zip(*indices_per_category)

train_index_all = []
test_index_all = []

for x in ss_all:
    train_index_this = np.concatenate([index_per_category[i][c[0]] for i, c in enumerate(x)])
    test_index_this = np.concatenate([index_per_category[i][c[1]] for i, c in enumerate(x)])
    #print(train_index_all.shape, train_index_all.dtype)
    assert np.unique(np.concatenate([train_index_this, test_index_this])).size == train_index_this.size + test_index_this.size
    #print(test_index_all.shape, test_index_all.dtype)
    categories_count_train_this = Counter(label_list[train_index_this]).values()
    #print(len(categories_count_train_this), max(categories_count_train_this), min(categories_count_train_this))
    assert len(categories_count_train_this)==num_class
    assert max(categories_count_train_this) == min(categories_count_train_this) == 15
    categories_count_test_this = Counter(label_list[test_index_this]).values()
    assert len(categories_count_test_this)==num_class
    assert max(categories_count_test_this) == min(categories_count_test_this) == 15
    train_index_all.append(train_index_this)
    test_index_all.append(test_index_this)
    
sss = zip(train_index_all, test_index_all)
for train_index, test_index in sss:
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    print(train_index.dtype, test_index.dtype)
# read in all images.
print('start reading...')
X = np.array([imread(f) for f in filelist])
print('end reading...')

# create a multiclass SVM
svm_step = OneVsRestClassifier(SVC(C=10,kernel='linear'), n_jobs=12) # C=10 to match paper behavior
normalize_step = StandardScaler()
pca_step = PCA()  # this will actually keep 3060 dimensions, which is number of training examples per time.
v1like_extraction_step = v1like.V1Like(pars_baseline='simple', n_jobs=12)
v1like_extraction_step_plus = v1like.V1Like(pars_baseline='simple_plus', n_jobs=12)
estimator = Pipeline([('v1s',v1like_extraction_step), ('normalize', normalize_step), ('pca', pca_step), ('svm', svm_step)])
estimator_plus = Pipeline([('v1s',v1like_extraction_step_plus), ('normalize', normalize_step), ('pca', pca_step), ('svm', svm_step)])

mean count 89.6470588235, std count 123.04109365, max count 800, min count 31
TRAIN: 1530 TEST: 1530
int64 int64
TRAIN: 1530 TEST: 1530
int64 int64
TRAIN: 1530 TEST: 1530
int64 int64
TRAIN: 1530 TEST: 1530
int64 int64
TRAIN: 1530 TEST: 1530
int64 int64
start reading...
end reading...


In [3]:
simple_performance = cross_val_score(estimator=estimator, X=X, y=label_list, cv=sss, n_jobs=1, verbose=0)
simple_performance_plus = cross_val_score(estimator=estimator_plus, X=X, y=label_list, cv=sss, n_jobs=1, verbose=0)
print(simple_performance)
print(simple_performance_plus)


working on problem of size 1530


[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    9.4s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:   22.3s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:   40.1s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:  1.0min
[Parallel(n_jobs=12)]: Done 624 tasks      | elapsed:  1.5min
[Parallel(n_jobs=12)]: Done 858 tasks      | elapsed:  2.1min
[Parallel(n_jobs=12)]: Done 1128 tasks      | elapsed:  2.7min
[Parallel(n_jobs=12)]: Done 1434 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done 1530 out of 1530 | elapsed:  3.6min finished
/home/yimengzh_everyday/miniconda2/envs/early-vision-toolbox/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/yimengzh_everyday/miniconda2/envs/early-vis


working on problem of size 1530


[Parallel(n_jobs=12)]: Done   2 tasks      | elapsed:    1.7s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    8.7s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:   20.3s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:   36.7s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:   59.0s
[Parallel(n_jobs=12)]: Done 624 tasks      | elapsed:  1.4min
[Parallel(n_jobs=12)]: Done 858 tasks      | elapsed:  1.9min
[Parallel(n_jobs=12)]: Done 1128 tasks      | elapsed:  2.6min
[Parallel(n_jobs=12)]: Done 1434 tasks      | elapsed:  3.2min
[Parallel(n_jobs=12)]: Done 1530 out of 1530 | elapsed:  3.4min finished



working on problem of size 1530


...


working on problem of size 1530


[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    9.5s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:   22.9s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:   41.5s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 624 tasks      | elapsed:  1.6min
[Parallel(n_jobs=12)]: Done 858 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 1128 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 1434 tasks      | elapsed:  3.7min
[Parallel(n_jobs=12)]: Done 1530 out of 1530 | elapsed:  4.0min finished
/home/yimengzh_everyday/miniconda2/envs/early-vision-toolbox/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/yimengzh_everyday/miniconda2/envs/early-vis


working on problem of size 1530


[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:    9.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:   22.0s
[Parallel(n_jobs=12)]: Done 264 tasks      | elapsed:   39.9s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done 624 tasks      | elapsed:  1.6min
[Parallel(n_jobs=12)]: Done 858 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 1128 tasks      | elapsed:  2.9min
[Parallel(n_jobs=12)]: Done 1434 tasks      | elapsed:  3.6min
[Parallel(n_jobs=12)]: Done 1530 out of 1530 | elapsed:  3.9min finished


[ 0.5875817   0.58431373  0.59150327  0.57973856  0.5869281 ]
[ 0.61764706  0.61045752  0.61895425  0.62026144  0.60915033]
